This is the code for the data extractor.

First, some maintenance work.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

#let python display all columns on output.
pd.set_option('display.max_columns', None)

File_Assember( ):
Read one or multiple concatenated files into a dataframe.

Input: file path.

Output: A pandas dataframe.

In [2]:
#set the file path, users can change to their own paths
path = r'C:\Users\Eric Zhong\Documents\2.INTERNSHIP\RESEARCH\iFiles_for_Leo'

#The function passes in the file path and file_no_, which means the number of file in order. 
#For example, we want to read the 5th txt file only: File_Assembler(path_, 5, 5)
#If we want to read files 1 - 7: File_Assembler(path_, 1, 7)
#in total 58 txt files for now

def File_Assembler(path_, file_lwr_, file_upper_):
    all_files = glob.glob(path_ + "/*.txt")
    count = 0
    #create an empty array, and use a for loop to append files to the array.
    li = []
    for filename in all_files:
        count = count + 1
        if count >= file_lwr_ and count <= file_upper_:
            x = pd.read_csv(filename, index_col=None, header=0, sep = '\t')
            li.append(x)
    Assembled_File = pd.concat(li, axis=0, ignore_index=True)
    return Assembled_File

File_Loader( )
Used when the user only wants to load one file as an alternative

INPUT:
    File path
    The number of file

OUTPUT:
    Dataframe containing the data


In [3]:
#Since now the scenario has changed, File_assembler is no longer needed at most times. A File_loader is written below
#in order to load single files more efficiently. User passes in the path and also the order number of file.
def File_Loader(path_, file_no_):
    all_files = glob.glob(path_ + "/*.txt")
    count = 0
    for filename in all_files:
        count = count + 1
        if count == file_no_:
            file_ = pd.read_csv(filename, index_col=None, header=0, sep = '\t')
    return file_

NONOX_data: A BIG function that reproduces the excel sheet's NONOX data, leaving only PMAX
Input
    Data from File_Assembler or File_Loader
    Time_alignment_constant, which is found to be 265 CAD in excel sheet 255
    TDC, EVO, EVC: 600, 128, 382 in excel sheet.
    NOoffset: The magical -390 constant when calculating NO average
    Engine cycle period: 720 for ours
Output
    Dataframe that contains all NONOX data in excel sheet.

In [7]:
def NONOX_data(Assembled_Data, time_alignment_constant_, TDC, EVO, EVC, NOoffset, Engine_Cycle_Period_):
#-------------------------------------------------tidy up the original dataframe-----------------------------------------
    #replace variable names for convenience
    a = Assembled_Data
    p = Engine_Cycle_Period_
    # drop all unneccessary columns, and the units row.
    a = a[['Crank Angle', 'NO', 'NOx']].drop([0])
    # change data types from strings to floats
    a = a.astype('float64')
    # drop all rows with 0 values
    a = a[a.NO != 0.0000]
    #reset the index and drop original index column
    a = a.reset_index().drop(['index'], axis=1)
    #set crank angle as the index
    a = a.set_index('Crank Angle')
    
#---------------------------------------------Generate basic data from existing data----------------------------------------
    # Create two columns for NONOX in ppm
    a['NO_ppm'] = a['NO'] * 200
    a['NOx_ppm'] = a['NOx'] * 200
    # Create NOx time aligned by shifting Nox_ppm values up by time constant amount 
    a['NOx_ppm_ta'] = a['NOx_ppm'].shift(-time_alignment_constant_)
    # Create NO2 column
    a['NO2'] = a['NOx_ppm_ta'] - a['NO_ppm']
    # Create NO2/NOx column
    a['NO2/NOx'] = a['NO2'] / a['NOx_ppm_ta']

#---------------------------------------------Generate Average NO, NOx since CAD 75000--------------------------------------
    # Fill the columns since CAD 75000
    x = 0
    while (x*p - NOoffset + TDC + EVC < a.shape[0]):
        a.loc[75000+x, 'NO_Avg'] = a['NO_ppm'].loc[75000 + x*p - NOoffset + TDC + EVO : 75000 + x*p - NOoffset + TDC + EVC].mean()
        a.loc[75000+x, 'NOx_Avg'] = a['NOx_ppm'].loc[75000 + x*p + TDC + EVO : 75000 + x*p + TDC + EVC].mean()
        x = x + 1
#--------------------------------------------NO2 avg and NO2/No avg--------------------------------------------------------
    a['NO2_Avg'] = a['NOx_Avg'] - a['NO_Avg']
    a['NO2/NO_Avg'] = a['NO2_Avg'] / a['NOx_Avg']

    return a

In [ ]:
#######################################Break the above function into small ones ############################################

Filter_NONOX( ):
    This function filters the original dataframe to contain only NO, NOx (V) indexed by crank angle

In [276]:
def Filter_NONOX(Assembled_Data):
#-------------------------------------------------tidy up the original dataframe-----------------------------------------
    #replace variable names for convenience
    a = Assembled_Data
    # drop all unneccessary columns, and the units row.
    a = a[['Crank Angle', 'NO', 'NOx']].drop([0])
    # change data types from strings to floats
    a = a.astype('float64')
    # drop all rows with 0 values
    a = a[a.NO != 0.0000]
    #reset the index and drop original index column
    a = a.reset_index().drop(['index'], axis=1)
    #set crank angle as the index
    a = a.set_index('Crank Angle')
    return a

TimeAlignmentNO2( )
With the user supplying NONOX filtered dataframe:
    This function adds columns for:
        NO NOX in ppm
        NOX timealigned
        NO2 and NO2/NOX
    

In [279]:
def TimeAlignmentNO2(Filter_NONOX_Output_, time_alignment_constant_):
#-------------------------------------------------tidy up the original dataframe-----------------------------------------
    #replace variable names for convenience
    a = Filter_NONOX_Output_
    
#---------------------------------------------Generate basic data from existing data----------------------------------------
    # Create two columns for NONOX in ppm
    a['NO_ppm'] = a['NO'] * 200
    a['NOx_ppm'] = a['NOx'] * 200
    # Create NOx time aligned by shifting Nox_ppm values up by time constant amount 
    a['NOx_ppm_ta'] = a['NOx_ppm'].shift(-time_alignment_constant_)
    # Create NO2 column
    a['NO2'] = a['NOx_ppm_ta'] - a['NO_ppm']
    # Create NO2/NOx column
    a['NO2/NOx'] = a['NO2'] / a['NOx_ppm_ta']
    return a

NONOX_Cyclic_Avg( ):
    A funtion that calculates the average of NO NOx within sampling window from CAD 75000 onwards, and generate NO2 NO2/NOX average.

INPUT:
    Output of TimeAlignmentNO2
    TDC EVO EVC
    NOoffset: the magical 390 in excel sheet 255
    Engine period of 720
OUTPUT: 
    After this function, one should be able to obtain a dataframe containing all NONOX data.

In [282]:
def NONOX_Cyclic_Avg(TimeAlignmentNO2_Output_, TDC, EVO, EVC, NOoffset, Engine_Cycle_Period_):
    #simplify variable name for convenience
    a = TimeAlignmentNO2_Output_
    p = Engine_Cycle_Period_
    x = 0
    while (x*p - NOoffset + TDC + EVC < a.shape[0]):
        a.loc[75000+x, 'NO_Avg'] = a['NO_ppm'].loc[75000 + x*p - NOoffset + TDC + EVO : 75000 + x*p - NOoffset + TDC + EVC].mean()
        a.loc[75000+x, 'NOx_Avg'] = a['NOx_ppm'].loc[75000 + x*p + TDC + EVO : 75000 + x*p + TDC + EVC].mean()
        x = x + 1
        
    a['NO2_Avg'] = a['NOx_Avg'] - a['NO_Avg']
    a['NO2/NO_Avg'] = a['NO2_Avg'] / a['NOx_Avg']

    return a

NO_NOx_plotter( )
Plots NO NOx data against crank angle.

Input:
    The NONOx filtered dataframe.
    Start and end CAD he wishes to plot, by default 0 - 4319999.
    Whether to plot NO, NOx or both
    y lower and upper limit, 0 and 9 by default, which is the range of the entire dataframe.

Output: NO NOX plot.

In [99]:
def NONOX_v_plotter(Filtered_data_, NOoffset_, Start_Crank_Angle_ = -360, End_Crank_Angle_ = 431640, plot_NO = True, plot_NOx = True, y_lower_limit_ = 0, y_upper_limit_ = 3):
    axes = plt.gca()
    a = Filtered_data_
    c = NOoffset_
    sca = Start_Crank_Angle_
    eca = End_Crank_Angle_
    yll = y_lower_limit_
    yul = y_upper_limit_
    #FROM EXCEL SHEET 255, CRANK ANGLE STARTS AT 75390, NO STARTS AT 75000
    a['NOshifted'] = a['NO'].shift(c)
    if plot_NO == True and plot_NOx == True:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted', label = 'NO', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx', ylim = (yll , yul), ax=axes)
    elif plot_NO == True and plot_NOx == False:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted', label = 'NO', ylim = (yll , yul), ax=axes)
    else:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx', ylim = (yll , yul), ax=axes)
    axes.set(xlabel="Crank Angle", ylabel="V")
    a.drop(columns=['NOshifted'])
    plt.show()
#as an example, we want to plot from "engine", from 205000 CAD to 215000 degree, the variation of NOx only:
#NO_NOx_plotter(engine, 205000, 215000, False, True)
#or, as False = 0, and True = 1:
#NO_NOx_plotter(engine, 205000, 215000, 0, 1)

In [140]:
def NONOX_ppm_plotter(Filtered_data_, NOoffset_, Start_Crank_Angle_ = -360, End_Crank_Angle_ = 431640, plot_NO = True, plot_NOx = True, plot_ratio = True, y_lower_limit_ = 0, y_upper_limit_ = 600, ylwrlmt_right = -0.5, yuprlmt_right = 0.5):
    axes = plt.gca()
    a = Filtered_data_
    c = NOoffset_
    sca = Start_Crank_Angle_
    eca = End_Crank_Angle_
    yll = y_lower_limit_
    yul = y_upper_limit_
    yllr = ylwrlmt_right
    yulr = yuprlmt_right
    #FROM EXCEL 255, IT IS SEEN THAT THE DATA ARE ACTUALLY REVERSED, WHY IS THAT?
    #i.e CRANK ANGLE STARTS AT 75000, NO and NO2/NOX STARTS AT 75390
    #The SHIFTING DIRECTION HAS BEEN REVERSED!
    a['NOshifted_ppm'] = a['NO_ppm'].shift(-c)
    a['NO2/NOxshifted'] = a['NO2/NOx'].shift(-c)
    if plot_NO == True and plot_NOx == True and plot_ratio == True:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted_ppm', label = 'NO_ppm', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx_ppm', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NO2/NOxshifted',secondary_y=True, label = 'NO2/NOx', ax=axes)
        axes.right_ax.set_ylim(yllr,yulr)
    elif plot_NO == True and plot_NOx == True and plot_ratio == False:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted_ppm', label = 'NO_ppm', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx_ppm', ylim = (yll , yul), ax=axes)
    elif plot_NO == True and plot_NOx == False and plot_ratio == True:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted_ppm', label = 'NO_ppm', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NO2/NOxshifted',secondary_y=True, label = 'NO2/NOx', ax=axes)
        axes.right_ax.set_ylim(yllr,yulr)
    elif plot_NO == True and plot_NOx == False and plot_ratio == False:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOshifted_ppm', label = 'NO_ppm', ylim = (yll , yul), ax=axes)
    elif plot_NO == False and plot_NOx == True and plot_ratio == True:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx_ppm', ylim = (yll , yul), ax=axes)
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NO2/NOxshifted',secondary_y=True, label = 'NO2/NOx', ax=axes)
        axes.right_ax.set_ylim(yllr,yulr)
    elif plot_NO == False and plot_NOx == True and plot_ratio == False:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NOx_ppm', ylim = (yll , yul), ax=axes)
    elif plot_NO == False and plot_NOx == False and plot_ratio == True:
        a.reset_index().reset_index().iloc[sca + 360 : eca + 360].plot(kind='line', x='index', y='NO2/NOxshifted',secondary_y=True, label = 'NO2/NOx', ax=axes)
        axes.right_ax.set_ylim(yllr,yulr)
    axes.set(xlabel="Crank Angle", ylabel="ppm")
    #a = a.drop(columns=['NOshifted_ppm', 'NO2/NOxshifted'])
    plt.show()
    return a
#as an example, we want to plot from "engine", from 205000 CAD to 215000 degree, the variation of NOx only:
#NO_NOx_plotter(engine, 205000, 215000, False, True)
#or, as False = 0, and True = 1:
#NO_NOx_plotter(engine, 205000, 215000, 0, 1)

Filter_Shifted( )
    The function aims to clean up and get rid of the extra shifted columns generated from the plots functions
    as I did not manage to find a way to stop the functions generating the extra columns
INPUT:
    The dataframe after plots, two boolean variables to indicate whether the user has used the two plot functions
    by default, the user is assumed to NOT have plotted, hence the function wont do anything

OUTPUT:
    Cleaned dataframe

In [151]:
def Filter_Shifted(Plots_Output, NO_v_plotted = False, NO_ratio_ppm_plotted = False):
    a = Plots_Output
    if NO_v_plotted == True and NO_ratio_ppm_plotted == True:
        a = a.drop(columns = ['NOshifted', 'NOshifted_ppm', 'NO2/NOxshifted'])
    elif NO_v_plotted == True and NO_ratio_ppm_plotted == False:
        a = a.drop(columns = ['NOshifted'])
    elif NO_v_plotted == False and NO_ratio_ppm_plotted == True:
        a = a.drop(columns = ['NOshifted_ppm', 'NO2/NOxshifted'])

    return a

Time( )
    This function calculates the average time taken after each crank angle, and allows user to input the starting crank angle.

In [3]:
def Time(dataframe, Start_Crank_Angle):
    a = dataframe
    s = Start_Crank_Angle
    Avg_Time = []
    for value in a.index:
        Avg_Time.append((360 + value)*0.04)
    a['Avg_Time'] = Avg_Time
    a['Avg_Time'] = a['Avg_Time'].shift(s + 360)
    return a

NONOX_separator( ):
A function that is able to separate data within sampling window and data out of sampling window.

Input:
    NONOX filtered dataframe
    Window width
    Window offset
    Engine cycle period
Output:
    Dataframe containing data within sampling window.
    Dataframe containing the other data.

In [ ]:
def NONOX_separator(NONOXDataframe_, Window_Width_, Offset_, Period_):
    Data_Within_Window_ = pd.DataFrame([], columns = ['Crank Angle', 'NO', 'NOx'])
    x = 0
    while (Offset_ + x * Period_ < NONOXDataframe_.shape[0]):
            samples = NONOXDataframe_.iloc[(Offset_ + x * Period_) : (Offset_ + x * Period_ + Window_Width_)]
            Data_Within_Window_ = Data_Within_Window_.append(samples)
            x = x + 1
    Data_Outside_Window_ = pd.DataFrame([], columns = ['Crank Angle', 'NO', 'NOx'])
    Data_Outside_Window_ = NONOXDataframe_[~NONOXDataframe_.index.isin(Data_Within_Window_.index)]
    
    return Data_Within_Window_, Data_Outside_Window_

In [ ]:
############################################## PRESSURE FUNCTIONS FROM NOW ON##############################################

Filter_PCYL1( ):

Input:
    Data from File_Assembler or File_Loader.

Output: 
    Dataframe that contains only Crank Angle, PCYL1.

In [4]:
def Filter_PCYL1(Assembled_Data):
    # drop all unneccessary columns, and the units row.
    Assembled_Data_filtered = Assembled_Data[['Crank Angle', 'PCYL1']].drop([0])
    # change data types from strings to floats
    Assembled_Data_filtered = Assembled_Data_filtered.astype('float64')
    #reset the index and drop original index column
    Assembled_Data_filtered = Assembled_Data_filtered.set_index(['Crank Angle'])
    #Assembled_Data_filtered = Assembled_Data_filtered.reset_index().drop(['index'], axis=1)
    #Make Crank Angle start at 0 instead of -360
    #Assembled_Data_filtered['Crank Angle'] = Assembled_Data_filtered['Crank Angle'] + 360

    return Assembled_Data_filtered

Psample()
    This function returns a dataframe that contains samples of PCYL1 per CAD
    As it requires stepping, the user should input his own Start_CAD, so only part of the large dataframe
    needs to be sampled

In [122]:
def Psample(P_Dataframe_, Start_CAD = -360):
    #set variable name for convenience
    P_sampled = pd.DataFrame([])
    a = P_Dataframe_
    x = 0
    #while x <= P_Dataframe_.index.value_counts().count() - 360 - Start_CAD:
    while x <= P_Dataframe_.index.max() - Start_CAD:
    #The sampling is only PSEUDO random, we can ensure the same sample taken by specifying random_state
    #Is this necessary though?
        sample = P_Dataframe_.loc[Start_CAD + x].sample(n = 1, random_state = 1)
        P_sampled = pd.concat([P_sampled, sample])
        x = x+1
    return P_sampled
        

PMAX( ):
Extracts cyclic maximum PCYL1.

INPUT:
    PCYL1 filtered dataframe
    Offset of engine cycle (how much CAD does the engine go through before the first recorded cycle)
    Engine cycle period
OUTPUT:
    A dataframe that contains maximum PCYL1 for every engine cycle

In [40]:
def PMAX(P_sampled, Start_Crank_Angle_ = -360, Period_ = 720):
    PMAX_ = pd.DataFrame([], columns = ['PMAX'])
    x = 0
    
    #Take one sample from the multiple Ps per degree.
    #The sampling is only PSEUDO random, we can ensure the same sample taken by specifying random_state
    #Is this necessary though?
    #P_group = (P_Dataframe_.groupby(['Crank Angle']))
    P_sample = P_group.apply(lambda y: y.sample(n = 1, random_state = 1))
    #P_sample = (P_Dataframe_.groupby(['Crank Angle'])).sample(n = 1, random_state = 1)
    while ((Start_Crank_Angle_ + x * Period_) < P_sample.shape[0]):
        #Find the max pressure value for every 720 degree cycle, append to the PMAX_ dataframe.
        PMAX_.loc[x+1] = list(P_sample.loc[x * Period_: (x + 1) * Period_ - 1].max())
        #increment x
        x = x + 1
    #Drop the last value of PMAX_, as there were originally 432001 records instead of 432000, need further clarification
    #PMAX_ = PMAX_.drop(PMAX_.shape[0])
    return PMAX_
#The output dataframe should have its index as the number of cycles.